In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from skopt.space import Real, Integer
from modules.interface import RunModel, RunOptimization

In [2]:
# --- INÍCIO DO SCRIPT DE TESTE ---

# 1. Carregar o Banco de Dados (California Housing)
print("1. Carregando o banco de dados...")
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target)

# 2. Dividir os dados em Conjunto de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- OTIMIZAÇÃO PARA LIGHTGBM ---
print("="*50)
print("INICIANDO OTIMIZAÇÃO PARA LIGHTGBM")
print("="*50)

space_lgbm = [
    Real(name='learning_rate', low = 0.01 , high = 0.9),
    Integer(name='num_leaves', low = 200, high = 700),
    Integer(name='max_depth', low = 200, high = 700),
    Real(name='feature_fraction', low = 0.5 , high = 1),
    Real(name='bagging_fraction', low = 0.7 , high = 1),
    Integer(name='bagging_freq', low = 1, high = 10),
    Real(name='lambda_l1',low = 0.0, high = 1),
    Real(name='lambda_l2',low = 0.5, high = 1)
]

fixed_params_lgbm = {
    'objective': 'mae', 
    'metric': 'mae', 
    'num_iterations ': 300,
    'random_state': 42, 
    'n_jobs': -1, 
    'verbose': -1
}

calibration_kwargs = {'callbacks':[lgb.early_stopping(stopping_rounds=10, verbose=False)]}
optimization_kwargs = {'n_initial_points': 15,
                        'n_calls': 30,
                        'initial_point_generator': 'lhs',
                        'random_state': 42,

                       
                       }

# Simplesmente passe 'model_type="lightgbm"'
optimizer_lgbm = RunOptimization()
results=optimizer_lgbm.run_multiple_optimizations(opt_class='gp_minimize',
    model_class="lgbm",
    datasets=[X_train, X_test, y_train, y_test],
    space=space_lgbm, 
    fixed_params=fixed_params_lgbm,
    metric='mae',
    calibration_kwargs=calibration_kwargs,
    optimization_kwargs= optimization_kwargs

)

print(f"\nMelhor MAE para LightGBM: {results[0].fun:.4f}")
print(f"Melhores parâmetros: {results[0].x}")

1. Carregando o banco de dados...
INICIANDO OTIMIZAÇÃO PARA LIGHTGBM
Optimization finished.
Best score (mean_absolute_error): 0.2818
Best parameters: {'learning_rate': 0.08032145496250154, 'num_leaves': np.int64(594), 'max_depth': np.int64(667), 'feature_fraction': 0.7414033489099505, 'bagging_fraction': 1.0, 'bagging_freq': np.int64(1), 'lambda_l1': 0.9148060866796668, 'lambda_l2': 0.5}

To train the final model with these parameters, call the '.fit_best_model()' method.

Melhor MAE para LightGBM: 0.2818
Melhores parâmetros: [0.08032145496250154, np.int64(594), np.int64(667), 0.7414033489099505, 1.0, np.int64(1), 0.9148060866796668, 0.5]


In [3]:
print(optimizer_lgbm.optimizer[0].y_test)


20046    0.47700
3024     0.45800
15663    5.00001
20484    2.18600
9814     2.78000
          ...   
15362    2.63300
16623    2.66800
18086    5.00001
2144     0.72300
3665     1.51500
Length: 4128, dtype: float64


In [4]:
# # Agora, para otimizar outro modelo, só mudamos o tipo e os parâmetros!
# optimizer_xgb = BayesianOptimization(
#     model_type="xgboost",
#     X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
#     space=space_xgb, fixed_params=fixed_params_xgb
# )
# resultado_xgb = optimizer_xgb.run(n_calls=15, random_state=42)
# print(f"\nMelhor MAE para XGBoost: {resultado_xgb.fun:.4f}")
# print(f"Melhores parâmetros: {resultado_xgb.x}")